In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
data = pd.read_csv('data.csv')

In [22]:
data['text'] = data['text'].str.lower()

/tmp/ipykernel_4739/369403302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].str.lower()


In [23]:
data.head()

,text
0,"defend democracy today: support fearless, insi..."
1,republican dan crenshaw goes down in flames du...
2,rep. dan crenshaw purchased stock during pande...
3,sen. ed markey applauded for silent dismissal ...
4,rep. dan crenshaw gets cornered on the view fo...


In [25]:
import re
from nltk.corpus import stopwords

In [26]:
## Removing special characters
data['text']=data['text'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
data['text']=data['text'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove any additional spaces
data['text']=data['text'].apply(lambda x: " ".join(x.split()))

/tmp/ipykernel_4739/2830895420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']=data['text'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
/tmp/ipykernel_4739/2830895420.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']=data['text'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
/tmp/ipykernel_4739/2830895420.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [9]:

from nltk.stem import WordNetLemmatizer

In [28]:
from nltk.stem import WordNetLemmatizer

lemmatizer=WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data['text']=data['text'].apply(lambda x:lemmatize_words(x))



/tmp/ipykernel_4739/3857269182.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']=data['text'].apply(lambda x:lemmatize_words(x))


In [29]:
data.head()

,text
0,defend democracy today support fearless insigh...
1,republican dan crenshaw go flame debate msnbcs...
2,rep dan crenshaw purchased stock pandemic fail...
3,sen ed markey applauded silent dismissal texas...
4,rep dan crenshaw get cornered view offering ri...


In [30]:
from sklearn.model_selection import train_test_split

In [31]:
x_train,x_test,y_train,y_test = train_test_split(data['text'],db[['target']],test_size=0.25,random_state=42)

In [32]:
x_train

3615    pollard defends spying u israel rare interview...
1721    usa need reckoning truth reconciliation actual...
576     joker harsh indictment neoliberalism gangster ...
985     huge victory black farmer hail 5b new covid re...
3775    newsmaxs list best gop secretary state u secre...
                              ...                        
1130    dignity work -- aocs green new deal -- save ne...
1294    ceo texas power grid operator terminated after...
860     stephen miller try attacking meghan prince har...
3507    woke mob target law professor china comment ut...
3174    real reason u cant win war anymore u soldier c...
Name: text, Length: 2933, dtype: object

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(x_train).toarray()
X_test_bow=bow.transform(x_test).toarray()

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(x_train).toarray()
X_test_tfidf=tfidf.transform(x_test).toarray()

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,recall_score,precision_score,f1_score,roc_auc_score,roc_curve

In [36]:
models={
    "Logisitic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "XGBoost": XGBClassifier()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train_tfidf, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train_tfidf)
    y_test_pred = model.predict(X_test_tfidf)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)


    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc


    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))
    
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))

    
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))

    
    print('='*35)
    print('\n')

/home/aarsh/arise/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logisitic Regression
Model performance for Training set
- Accuracy: 0.9417
- F1 score: 0.9410
- Precision: 0.9834
- Recall: 0.8671
- Roc Auc Score: 0.9288
----------------------------------
Model performance for Test set
- Accuracy: 0.8671
- F1 score: 0.8615
- Precision: 0.9349
- Recall: 0.6835
- Roc Auc Score: 0.8280




/home/aarsh/arise/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 1.0000
- Recall: 0.9983
- Roc Auc Score: 0.9991
----------------------------------
Model performance for Test set
- Accuracy: 0.8793
- F1 score: 0.8731
- Precision: 0.9878
- Recall: 0.6779
- Roc Auc Score: 0.8365


Decision Tree
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 1.0000
- Recall: 0.9983
- Roc Auc Score: 0.9991
----------------------------------
Model performance for Test set
- Accuracy: 0.8640
- F1 score: 0.8643
- Precision: 0.8077
- Recall: 0.8235
- Roc Auc Score: 0.8554


XGBoost
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 1.0000
- Recall: 0.9983
- Roc Auc Score: 0.9991
----------------------------------
Model performance for Test set
- Accuracy: 0.9407
- F1 score: 0.9401
- Precision: 0.9572
- Recall: 0.8768
- Roc Auc Score: 0.9271


